In [1]:
def dist(a,b):   
    # la distance euclidienne entre a et b
    resultat = np.array([])
    for vec in b:
        resultat = np.append(resultat,[(np.sqrt(np.sum(np.power(a-vec, 2))))])
    return resultat